# 03 - Merge With Hashing Info

In [ ]:
from libraries import *
from parameters import *

from pathlib import Path
from tqdm.auto import tqdm

In [ ]:
%load_ext rpy2.ipython

In [ ]:
os.getcwd()
os.chdir(projectDir)

***

In [ ]:
adata = sc.read(par_save_filename_1)

In [ ]:
hash_ads = {}
feats = {}

for d in tqdm(sorted([x for x in Path('data').iterdir() if x.is_dir() and x.name.startswith('E')])):
    sample = d.name.replace('_', '-')
    roundlow = sample.split('-')[0].lower()
    chrom = sample.split('-')[1].lower()
    
    hash_path = Path('data') / sample.replace('-', '_') / f'{roundlow}_ch{chrom}_mixhash' / f'{roundlow}_ch{chrom}_mixhash_demux.h5ad'
    if hash_path.is_file():
        print(sample)
        hash_ads[sample] = sc.read(hash_path)
        
    feat_path = Path(f'data/{sample.replace("-", "_")}/R{roundlow[1:]}_Ch{chrom}_FeatBarc/R{roundlow[1:]}_Ch{chrom}_FeatBarc.csv')
    if feat_path.is_file():
        print(sample)
        feats[sample] = pd.read_csv(feat_path)

## Merge hash IDs

In [ ]:
%%time

hash_ad = sc.AnnData.concatenate(*hash_ads.values(), 
                                 batch_key='mixhash', 
                                 batch_categories=list(hash_ads.keys()))
hash_ad

In [ ]:
hash_ad.obs.index = ['-'.join([x.split('-')[0], 
                               '1', 
                               x.split('-')[1], 
                               x.split('-')[2]]) for x in hash_ad.obs.index]

In [ ]:
adata.obs['demux_type'] = hash_ad.obs.demux_type
adata.obs['hash_assignment'] = hash_ad.obs.assignment

In [ ]:
adata.obs['hash_assignment'] = adata.obs['hash_assignment'].astype('category')

Filter out doublets and multiplets


In [ ]:
adata = adata[adata.obs.demux_type == 'singlet']

In [ ]:
adata.write(par_save_filename_2)